In [ ]:
import os
import glob
import math
import random
import pickle
from collections import defaultdict

In [ ]:
folder = './train'

image_folder = glob.glob(os.path.join(folder,'images','*'))
print(len(image_folder))
print(image_folder[:5])

In [ ]:
total_samples = defaultdict(int)

for image_folder_path in image_folder:
    *_,file = image_folder_path.split('/')
    number,gender,race,age = file.split('_')
    
    label = 0
    age = int(age)
    
    if age>=30 and age<60:
        label+=1
    elif age>=60:
        label+=2
    
    if gender == 'female':
        label+=3
        
    total_samples[label]+=1

print(total_samples)

In [ ]:
valid_folder_names_per_fold = defaultdict(list) # fold 별 folder name
train_folder_names_per_fold = defaultdict(list)

valid_labels_per_fold = defaultdict(list) # fold 별 labels 정보
train_labels_per_fold = defaultdict(list)
class_count_per_fold = defaultdict(lambda: [0 for _ in range(6)]) # fold별 class 카운트 정보

shuffle = True
k = 5

if shuffle:
    random.shuffle(image_folder)


for image_folder_path in image_folder:
    *_,file = image_folder_path.split('/')
    number,gender,race,age = file.split('_')
    
    # label calcuate # 
    label = 0
    age = int(age)
    
    if age>=30 and age<60:
        label+=1
    elif age>=60:
        label+=2
    
    if gender == 'female':
        label+=3
    #################
    
    flag = False
    check_point = -1
    
    for i in range(k):
        # ex ) k=5 , 9 -> (2,2,2,2,1) , 13 -> (3,3,3,2,2)
        if class_count_per_fold[i][label]<(total_samples[label]//k): # 폴드별 라벨 카운트 (20%씩 들어감)
            class_count_per_fold[i][label]+=1
            valid_labels_per_fold[i].append(label)
            valid_folder_names_per_fold[i].append(image_folder_path)
            flag = True
            check_point = i
            break
    
    if flag:
        for i in range(k):
            if i!= check_point:
                train_labels_per_fold[i].append(label)
                train_folder_names_per_fold[i].append(image_folder_path)
    
    if not flag:
        # 13 -> 위에 코드에선 (2,2,2,2,2) 를 넣어줌. 나머지를 (3,3,3,2,2)로 넣어주기 위함 
        for i in range(k):
            if class_count_per_fold[i][label]<=(total_samples[label]//k):
                class_count_per_fold[i][label]+=1
                valid_labels_per_fold[i].append(label)
                valid_folder_names_per_fold[i].append(image_folder_path)
                flag = True
                break
        
        if flag:
            for i in range(k):
                if i!= check_point:
                    train_labels_per_fold[i].append(label)
                    train_folder_names_per_fold[i].append(image_folder_path)

In [ ]:
print(class_count_per_fold)

In [ ]:
with open('train_folder_names_per_fold.pkl', 'wb') as f:
    pickle.dump(train_folder_names_per_fold, f)

with open('valid_folder_names_per_fold.pkl', 'wb') as f:
    pickle.dump(valid_folder_names_per_fold, f)


    
with open('train_labels_per_fold.pkl', 'wb') as f:
    pickle.dump(train_labels_per_fold, f)
    
with open('valid_labels_per_fold.pkl', 'wb') as f:
    pickle.dump(valid_labels_per_fold, f)

In [ ]:
train_image_list = glob.glob(os.path.join('./train','images','*/*.*'))

label_list = []

for train_path in train_image_list:
    _,_,_,path,file_name = train_path.split('/')
    number,gender,race,age = path.split('_')
    mask, _ = file_name.split('.')
    
    label = 0
    
    age = int(age)
    if age>=30 and age<60:
        label+=1
    elif age>=60:
        label+=2
    
    if gender == 'female':
        label+=3
    
    if mask == 'incorrect_mask':
        label+=6
    elif mask == 'normal':
        label+=12
        
    label_list.append(label)


print(label_list[:20])

In [ ]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from collections import Counter

image,label = train_image_list,label_list

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

image = np.array(image)
label = np.array(label)

valid_folder_names_per_fold = defaultdict(list) # fold 별 folder name
train_folder_names_per_fold = defaultdict(list)

valid_labels_per_fold = defaultdict(list) # fold 별 labels 정보
train_labels_per_fold = defaultdict(list)

for i,(train_idx, test_idx) in enumerate(skf.split(image, label)):
    a = Counter(label[train_idx])
    b = Counter(label[test_idx])
    train_folder_names_per_fold[i] = image[train_idx]
    valid_folder_names_per_fold[i] = image[test_idx]
    
    train_labels_per_fold[i] = label[train_idx]
    valid_labels_per_fold[i] = label[test_idx]

In [ ]:
print(train_folder_names_per_fold)

In [ ]:
with open('sklearn_train_folder_names_per_fold.pkl', 'wb') as f:
    pickle.dump(train_folder_names_per_fold, f)

with open('sklearn_valid_folder_names_per_fold.pkl', 'wb') as f:
    pickle.dump(valid_folder_names_per_fold, f)

    
with open('sklearn_train_labels_per_fold.pkl', 'wb') as f:
    pickle.dump(train_labels_per_fold, f)
    
with open('sklearn_valid_labels_per_fold.pkl', 'wb') as f:
    pickle.dump(valid_labels_per_fold, f)